In [ ]:
!pip install langchain -q
!pip install openai -q

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'LANGCHAIN_KEY'

In [ ]:
!pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/short stories.pdf")
loader

In [6]:
pages = loader.load_and_split()
pages

[Document(page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short\nstory’ of about five hundred words to longer and\nmore complex works. The longer ones, with their\nstatus of middle length, fall between the tautness\nof the short narrative and the expansiveness of\nthe novel.\nThere can be thematic variation too. The stories\ndeal with fantasy, reality, alienation and the\nproblem of choice in personal life. There are three\nshort stories and two long ones in this section\nrepresenting writers from five cultures.\n20

In [7]:
pages[0].page_content

'1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short\nstory’ of about five hundred words to longer and\nmore complex works. The longer ones, with their\nstatus of middle length, fall between the tautness\nof the short narrative and the expansiveness of\nthe novel.\nThere can be thematic variation too. The stories\ndeal with fantasy, reality, alienation and the\nproblem of choice in personal life. There are three\nshort stories and two long ones in this section\nrepresenting writers from five cultures.\n2024-25'

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " "
    ],
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(pages)

In [9]:
chunks[0]

Document(page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of', metadata={'source': '/content/short stories.pdf', 'page': 0})

In [10]:
count=0
for chunk in chunks:
  if count<5:
    print(len(chunk.page_content))
  else:
    break
  count+=1

456
480
195
450
459


In [11]:
[chunks[0]]

[Document(page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of', metadata={'source': '/content/short stories.pdf', 'page': 0})]

In [ ]:
!pip install faiss-gpu

In [ ]:
!pip install sentence-transformers

In [ ]:
from langchain_community.vectorstores import FAISS
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [15]:
vectorstore = FAISS.from_documents(chunks,embeddings)
#faiss_index = FAISS.from_documents(chunks, OpenAIEmbeddings())

In [16]:
docs = vectorstore.similarity_search("What are we offering to the customer?", k=4)
print(docs[0])
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

page_content='10/KALEIDOSCOPE\nLanguage W ork\nA.Vocabulary\nLook up the meanings of the following phrases under ‘dream’\nand ‘sell’ in the dictionary\ndream         sell\ndream on sell-by date\ndream something away selling-point\n(not) dream of doing something sell-out\ndream something up selling price\nlook like a dream seller’s market\nB.Grammar: Emphasis\nRead this sentence carefully\nOne morning  at nine o’clock, while we were having\nbreakfast on the terrace of the Havana Riviera Hotel' metadata={'source': '/content/short stories.pdf', 'page': 9}
9: 10/KALEIDOSCOPE
Language W ork
A.Vocabulary
Look up the meanings of the following phrases under ‘dream’
and ‘sell’ in the dictionary
dream         sell
dream on sell-by date
dream something away selling-point
(not) dream of doing something sell-out
dream something up selling price
look like a dream 
3: 4/KALEIDOSCOPE
paradise of black marketeering and international espionage.
I could not have imagined a more suitable spot for my
fugit

In [17]:
docs_with_score = vectorstore.similarity_search_with_score("What are we doing for the customer?", k=3)
docs_with_score[0]

(Document(page_content='One morning  at nine o’clock, while we were having breakfast\non the terrace of the Havana Riviera Hotel under a bright\nsun, a huge wave picked up several cars that were driving\ndown the avenue along the seawall or parked on the\npavement, and embedded one of them in the side of the\nhotel. It was like an explosion of dynamite that sowed panic\non all twenty floors of the building and turned the great\nentrance window to dust. The many tourists in the lobby', metadata={'source': '/content/short stories.pdf', 'page': 1}),
 1.6010599)

Create a context aware LLM Chain

In [37]:
#create chain for documents

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

llm = ChatOpenAI()

template = """Answer the following question based on the provided context
<context>
{context}
</context>

Question:{input}
"""

#This creates a chain that will combine documents and use the provided template and language model to generate responses.
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm,prompt)

# Example usage
context = "The sun rises in the east and sets in the west."
question = "Which direction does the sun set?"

# Convert context into a Document object
documents = [Document(page_content=context)]

# Get the answer from the document chain
answer = document_chain.invoke({"input": question, "context": documents})
print(answer)



The sun sets in the west.


In [38]:
#Now let's use this in our scenario

document_chain.invoke({
    "input":"What is the story about?",
    "context":[chunks[0]]
})

"The story is about a prose narrative of limited length that organizes the action and thoughts of its characters into a plot. The plot can be comic, tragic, romantic, or satiric, with the central incident selected to manifest the protagonist's life and character. The details in the story contribute to the development of the plot."

###Retriver based Q&A system: Create Retrival Chain (RAG)
The system retrieves relevant documents from a vector store and then uses a language model to generate an answer based on the retrieved documents.

RAG stands for Retrieval-Augmented Generation. This means that we will enrich the prompt that we send to the LLM. We will use with the documents that wil will retrieve from the vector store for this. LangChain comes with the function create_retrieval_chain that allows you to create one of these.

In [20]:
from langchain.chains import create_retrieval_chain

#This method converts a vector store into a retriever object
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

#This function creates a chain that first uses the retriever to find relevant documents
#and then processes these documents using the document_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

response = retrieval_chain.invoke({
    "input": "What is the story about?"
})
response

{'input': 'What is the story about?',
 'context': [Document(page_content='development of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short\nstory’ of about five hundred words to longer and\nmore complex works. The longer ones, with their\nstatus of middle length, fall between the tautness\nof the short narrative and the expansiveness of\nthe novel.\nThere can be thematic variation too. The stories\ndeal with fantasy, reality, alienation and the\nproblem of choice in personal life. There are three', metadata={'source': '/content/short stories.pdf', 'page': 0}),
  Document(page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\nc

In [21]:
response['answer']

'The story is about the development of the plot in short stories, including the organization of action and thoughts of characters into a plot, as well as the thematic variations that can be found in short stories, such as fantasy, reality, alienation, and the problem of choice in personal life.'

####Converstaional Retrival Chain : RAG Chain
This type of chain is designed to take into account the history of the conversation when generating search queries and retrieving relevant documents. Let’s go through each part step by step.

Now we will create exactly the same thing as above, but we will have the AI assistant take the history of the conversation into account. In short, we will build the same chain as above but with we will take into account the previous messages in these two steps of the chain:

- When fetching the documents from the vector store. We will fetch documents related to the entire conversation and not just the latest message.
- When answering the question. We will send to the LLM the history of the conversation along the context and query.



In [22]:
#create a retriever that is aware of the conversation history
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

history_retriever_chain = create_history_aware_retriever(llm,retriever,prompt)


In [40]:
#Simple Example:

#Define a more complex prompt for answering questions based on retrieved documents:
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

#Create the document processing chain
document_chain = create_stuff_documents_chain(llm, answer_prompt)

#Create the final conversational retrieval chain
conversational_retrieval_chain = create_retrieval_chain(history_retriever_chain, document_chain)

#Define conversation history
chat_history = [
    HumanMessage(content="What is climate change?"),
    AIMessage(content="Climate change refers to long-term changes in temperature, precipitation, and other atmospheric conditions on Earth.")
]

#Invoke the conversational retrieval chain
response = conversational_retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "How does it affect polar bears?"
})


In [41]:
response

{'chat_history': [HumanMessage(content='What is climate change?'),
  AIMessage(content='Climate change refers to long-term changes in temperature, precipitation, and other atmospheric conditions on Earth.')],
 'input': 'How does it affect polar bears?',
 'context': [Document(page_content='of the story.\nLet us rewrite th e sentence, beginning with ‘a huge wave’ and\nthe first part following ‘hotel’ and  notice the difference in the\neffect.\nA huge wave picked up several cars that were driving\ndown the avenue along the seawall or parked on the\npavement, and embedded one of them in the side of\nthe hotel, one morning  at nine o’clock, while we were\nhaving breakfast on the terrace of the Havana Riviera\nHotel under a bright sun.\n2024-25', metadata={'source': '/content/short stories.pdf', 'page': 9}),
  Document(page_content='and sealed of f the gate to the sea and installed another ,\n1 11 11\nGabriel Garcia Marquez\n1927-2014\n2024-25', metadata={'source': '/content/short stories.pd

In [42]:
response['answer']

'Climate change affects polar bears in various ways. One of the main impacts is the loss of their sea ice habitat, which is crucial for hunting seals, their main food source. As the Arctic sea ice melts due to rising temperatures, polar bears have to travel longer distances to find food, leading to increased energy expenditure and decreased reproductive success. Additionally, the loss of sea ice also exposes polar bears to more human activity and potential conflicts.'

In [43]:
from langchain_core.messages import HumanMessage,AIMessage

chat_history = [
    HumanMessage(content="Is this a story?"),
    AIMessage(content="Yes!")
]

history_retriever_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me more about it!"
})

[Document(page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of', metadata={'source': '/content/short stories.pdf', 'page': 0}),
 Document(page_content='development of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short\nstory’ of about five hundred words to longer and\nmore complex works. The longer ones, with their\nstatus of middle length, fall between the tautness\nof the short narrative and the expansiveness of\nthe novel.\nThere can be thematic variation too. The stories\ndeal with fantasy, reality, 

In [24]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(history_retriever_chain, document_chain)

In [44]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "Is this a story?"
})

In [45]:
response['answer']

'No, this is not a story itself but an introduction to a collection of short stories.'

In [46]:
# simulate conversation history

chat_history = [
    HumanMessage(content="Is this a collection of stories?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [47]:
response['answer']

"This is a collection of short stories that covers a range of prose fiction, from very short to longer and more complex works. The stories in this collection deal with themes such as fantasy, reality, alienation, and the problem of choice in personal life. There are three short stories and two long ones in this section representing writers from five cultures. The stories are organized into a plot structure that may be comic, tragic, romantic, or satiric, with the central incident selected to manifest the protagonist's life and character. The details in the stories contribute to the development of the plot."